In [ ]:
import math
import numpy as np
import pandas as pd

qtInter,qtOut,qtInput,e,alpha,epoca = 50,10,784,0,0.5,0

dataset = pd.read_csv('mnist_test.csv')
epoca=0
matrizErros=[]
v = np.random.uniform(-1,1,(qtInput,qtInter))
w = np.random.uniform(-1,1,(qtInter,qtOut))

In [ ]:
def algoritmo(isTreino,qtEpoca):
    global epoca
    for i in range (qtEpoca):
        readLine(dataset,isTreino)
        epoca+=1

In [ ]:
def readLine(dSet,isTreino):
    global e
    r = 0
    e = 0
    for row in dSet.itertuples(index=False):
        linha = list(row)
        target = targetVetor(linha.pop(0))
        linha = trataLinha(linha)
        forward(linha,v,isTreino,target,r)
        r+=1
    matrizErros.append(e,r,epoca)

In [ ]:
def forward(linha,v,isTreino,target,r):
    global e
    inZ = inputZ(linha,v)
    Z = funcAtivacao(inZ,'relu')
    inY = inputY(Z,w)
    Y = funcAtivacao(inY,'relu')
    print (Y)
    if isTreino:
        if (not verificaAcerto(target,Y)):
            e+=1
            backPropagation(inZ,Z,inY,Y,target,linha)
    print (f'Linha: {r} Epoca: {epoca}')
    print (f'Erros: {e}')
    print (' _____________________________________')

In [ ]:
def backPropagation(inZ,Z,inY,Y,target,linha):
    delK =[]
    inJ  =[]
    delJ =[]
    delK = deltaK(target,Y,inY,'relu') 
    inJ  = deltainJ(delK,w)
    delJ = deltaJ(inJ,inZ,'relu')
    atualizaPesos(delK,Z,linha,delJ)

In [ ]:
def trataLinha(inp):
    for i in range(len(inp)):
        if (inp[i]>0):
            inp[i] = inp[i]/255
        else:
            inp[i] = inp[i]
    return inp

def targetVetor(labels, num_classes=10):
    retTarget = np.eye(num_classes)[labels]
    return retTarget

def inputZ(inp,pesos):
    soma = 0
    inX = np.zeros(qtInter)
    for j in range (qtInter):
        for i in range (qtInput):
            soma +=(inp[i]) * pesos[i][j]
        inX[j] = soma
        soma = 0
    return inX

def inputY(inp,pesos):
    soma = 0
    inX = np.zeros(qtOut)
    for k in range (qtOut):
        for j in range (qtInter):
            soma +=(inp[j]) * pesos[j][k]
        inX[k] = soma
        soma = 0
    return inX

def funcAtivacao(x,func) :
    if (func == 'relu') : 
        return np.maximum(x,0) 
    elif (func == 'sig') : 
        return sigmoide(x)
    else: 
        print('parametro invalido!')

def derivada(x,func):        
    if (func == 'relu') : 
        return derivRelu(x) 
    elif (func == 'sig') : 
        return derivSig(x)
    else: 
        print('parametro invalido!')
        
def tanh(x):
    return np.sinh(x) / np.cosh(x)

def sigmoide(x):
    sig = np.zeros(len(x))
    for i in range(len(x)):
        sig[i] = (1/1+ np.exp(-x[i]))
    return sig

def verificaAcerto(tar,out):
    for i in range (qtOut):
        if (tar[i] != out[i]):
            return False
    return True

def deltaK(targetK,Yk,YinK,func) :
    dK = np.zeros(qtOut)
    for i in range (qtOut) :
        erro = targetK[i] - Yk[i]
        deriv = derivada(YinK[i],func)
        dK[i] = erro * deriv
    return dK

def deltainJ (dK,peso):
    inJ = np.zeros(qtInter)
    for k in range (qtOut) :
        for j in range (qtInter):
            inJ[j] += dK[k] * peso[j][k]
    return inJ

def deltaJ(inJ,inZ,func):
    delJ = np.zeros(qtInter)
    for j in range (qtInter):
        deriv = derivada(inZ[j],func)
        delJ[j] = inJ[j] * deriv
    return delJ

def atualizaPesos(delK,Z,linha,delJ):
    global w
    global v
    w = deltaW(delK,Z)
    v = deltaV(linha,delJ)
    print (w)
    
def deltaW(dK,Ze):
    dW = np.zeros((qtInter,qtOut))
    for j in range(qtInter):
        for k in range(qtOut):
            dW[j][k] = alpha * dK[k] * Ze[j]
    return dW

def deltaV(linha,dJ):
    qtInput = len(linha)
    dV = np.zeros((qtInput,qtInter))
    for i in range(qtInput):
        for j in range (qtInter):
            dV[i][j] = alpha * dJ[j] * linha[i]
    return dV
    
def derivRelu(x):
    if (x<0):
        return 0
    elif (x>=0):
        return 1

In [ ]:
algoritmo(True,1)